In [ ]:
##### import pickle
import random
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import os
import random
import datetime
import h5py
import sklearn.metrics 
import cv2
import skimage.measure as measure
import skimage.filters as filters
import skimage.morphology as morphology
import skimage.exposure as exposure
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.compat.v2 as tf
import seaborn as sns
import matplotlib.patches as patches
import pickle
from functions import *

In [ ]:
#os.environ["CUDA_VISIBLE_DEVICES"]="1" # NVIDIA GeForce RTX 3090
os.environ["CUDA_VISIBLE_DEVICES"]="3" # NVIDIA GeForce RTX 2080
 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
gpus = tf.config.experimental.list_physical_devices('GPU')
 
print(gpus)

# Test set

In [ ]:
directory = "Data/Testset_Lavendel"
subdir = "Lavendel_good"
path_dir = os.path.join(directory, subdir)

with open(os.path.join(path_dir,'TEST_Images_Lavendel.pkl'), 'rb') as file:
    test_images = pickle.load(file)

with open(os.path.join(path_dir,'TEST_Ground_truth_Lavendel.pkl'), 'rb') as file:
    test_ground_truth = pickle.load(file)

with open(os.path.join(path_dir,'TEST_Genotypes_Lavendel.pkl'), 'rb') as file:
    test_genotypes = pickle.load(file)
    
name_test_prediction = "Lavendel_good"
path_evaluation = "Results/Evaluation robustness final model"
path_test_prediction = os.path.join(path_evaluation, name_test_prediction)
with open(path_test_prediction, 'rb') as file:
    test_predictions = pickle.load(file)

In [ ]:
test_genotypes

## Data exploration

In [ ]:
print(test_images.dtype)

In [ ]:
print("The test set contains", str(len(test_images)), "images.")
print("The length of the list with chromosome numbers is:", str(len(test_ground_truth)))
print("The length of the list with genotypes is:", str(len(test_genotypes)))

In [ ]:
print("The images in the test set have a width of", str( test_images.shape[2]),"and a height of",str(test_images.shape[1]))

In [ ]:
print("The maximum pixel value of the images is:", str(np.amax(test_images)))
print("The minumum pixel value of the images is:", str(np.amin(test_images)))

### Plot

In [ ]:
i = random.choice(range(len(test_images)))
plt.figure(figsize=(15,10))
plt.imshow(test_images[i],cmap="gray")
plt.axis("off")

# Prediction

## Final model

In [ ]:
name_model = "Final (M0_6(3x3)_2)"
filepath_dic = "Results/" + name_model
filepath_checkpoint_model = filepath_dic + "/checkpoint.model.keras"

## Prediction

In [ ]:
model_best = tf.keras.models.load_model(filepath_checkpoint_model)
batch_size = 1
test_predictions = model_best.predict(
    test_images,
    batch_size=batch_size)

In [ ]:
test_predictions.shape

In [ ]:
name_test_prediction = "Lavendel_good"
path_evaluation = "Results/Evaluation robustness final model"
path_test_prediction = os.path.join(path_evaluation, name_test_prediction)
test_predictions = test_predictions.reshape((84,2048,2688))
with open(path_test_prediction, 'wb') as file:
    pickle.dump(test_predictions, file)

In [ ]:
i = random.choice(range(len(test_images)))
plt.figure(figsize=(50,25))
plt.subplot(1,2,1)
plt.imshow(test_images[i],cmap="gray")
plt.axis("off")
plt.title("Microscopic mage", size=50)
plt.subplot(1,2,2)
plt.imshow(test_predictions[i],cmap="nipy_spectral")
plt.axis("off")
plt.title("Prediction", size=50)

# Post-processing

## Parameters for post-processing

In [ ]:
blurring = True
blur_kernel_size = 11

kernel_shape = "ellipse"

threshold_technique = cv2.THRESH_BINARY+cv2.THRESH_OTSU
threshold = 1

morphological_operations = {"erosion":{"kernel_size":11, "iterations":1},
                            "closing":{"kernel_size":11, "iterations":1}}

order_morphological_operations =  ["closing","erosion"] 

## Post-processing

In [ ]:
post_processed_predictions = []

for prediction in test_predictions:
    
    # normalize pixel values between 0 and 1
    normalized = normalization(prediction)
    normalized = (normalized*255).astype("uint8")
    
    # post-processing
    post_processed  = post_processing(
        im=normalized,
        blurring=blurring,
        blur_kernel_sz=blur_kernel_size,
        thresh_technique=threshold_technique,
        thresh=threshold,
        kernel_shape=kernel_shape,
        morph_ops=morphological_operations,
        order_morph_ops=order_morphological_operations)
    
    post_processed_predictions.append(post_processed)

### Figures

In [ ]:
index = 43

plots = []

    
prediction = test_predictions[index]
plots.append(prediction)

# normalisation
normalised = normalization(prediction)
normalised = (normalised*255).astype("uint8")
plots.append(normalised)
    
# blurring
blur_kernel = np.ones((11, 11), np.float32)/11**2
blurred = cv2.filter2D(src=normalised, ddepth=-1, kernel=blur_kernel)
plots.append(blurred)
 
# thresholding
binary = cv2.threshold(blurred,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
plots.append(binary)
        

# morpholoical operations
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11, 11))
closing = cv2.morphologyEx(binary, cv2.MORPH_CLOSE,kernel, iterations=1)
erosion = cv2.erode(closing, kernel, iterations=1)
plots.append(closing)
plots.append(erosion)
        
# labbeling
labelled = morphology.label(erosion)
plots.append(labelled)

cmaps = ["nipy_spectral", "nipy_spectral", "nipy_spectral", "gray", "gray", "gray", "nipy_spectral"]
# figure
plt.figure(figsize=(100,80))
for i,plot in enumerate(plots):
    plt.subplot(1,len(plots),i+1)
    plt.imshow(plot, cmap=cmaps[i], interpolation='none')
    plt.axis("off")

In [ ]:
plt.imsave(os.path.join("Images thesis/Post-processing","Prediction "+str(index)+".png"), arr=prediction, cmap="nipy_spectral", format="png")
plt.imsave(os.path.join("Images thesis/Post-processing","Average blurring "+str(index)+".png"), arr=blurred, cmap="nipy_spectral", format="png")
plt.imsave(os.path.join("Images thesis/Post-processing","Thresholding "+str(index)+".png"), arr=binary, cmap="gray", format="png")
plt.imsave(os.path.join("Images thesis/Post-processing","Closing "+str(index)+".png"), arr=closing, cmap="gray", format="png")
plt.imsave(os.path.join("Images thesis/Post-processing","Erosion "+str(index)+".png"), arr=erosion, cmap="gray", format="png")
plt.imsave(os.path.join("Images thesis/Post-processing","Labelling "+str(index)+".png"), arr=labelled, cmap="nipy_spectral", format="png")

# Evaluation

In [ ]:
textwidth = 455.24411

def set_size(width, fraction=1):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float
            Document textwidth or columnwidth in pts
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim

## Chromosome number

In [ ]:
dict_chromosome_number = {"Genotype":[], "Actual chromosome number":[], "Predicted chromosome number":[]}
for i,post_processed_prediction in enumerate(post_processed_predictions):
    genotype = test_genotypes[i]
    n_chromosomes_real = test_ground_truth[i]
    n_chromosomes_predicted = len(centroid(post_processed_prediction))
    
    dict_chromosome_number["Genotype"].append(genotype)
    dict_chromosome_number["Actual chromosome number"].append(n_chromosomes_real)
    dict_chromosome_number["Predicted chromosome number"].append(n_chromosomes_predicted)

### Figure

In [ ]:
i = random.choice(range(len(test_images)))
plt.figure(figsize=(50,20))
plt.subplot(1,3,1)
plt.imshow(test_images[i],cmap="gray")
plt.axis("off")
plt.title("Microscopic image " + str(i) + "-" + str(actual_numbers[i]), size=50)
plt.subplot(1,3,2)
plt.imshow(test_predictions[i], cmap="nipy_spectral")
plt.axis("off")
plt.title("Prediction", size=50)
plt.subplot(1,3,3)
plt.imshow(post_processed_predictions[i], cmap="nipy_spectral")
plt.axis("off")
plt.title("Post-processed prediction" + "-" + str(predicted_numbers[i]), size=50)

### Scatterplot

In [ ]:
fig, ax = plt.subplots(figsize=(set_size(textwidth)[0]*2, set_size(textwidth)[1]*2))
sns.scatterplot(data=dict_chromosome_number, 
                x="Actual chromosome number",
                y="Predicted chromosome number",
                hue="Genotype",
                ax=ax,
               s=100)
ax.tick_params(axis='both', which='major', labelsize=24)
ax.set_xlabel("Werkelijke chromosoomaantal", fontsize=24)
ax.set_ylabel("Voorspelde chromosoomaantal", fontsize=24)
n = np.linspace(0, max(max(dict_chromosome_number["Actual chromosome number"]), max(dict_chromosome_number["Predicted chromosome number"])), 1000)
ax.plot(n, n, 'k-')
ax.legend(fontsize=18)
legend_labels = ax.get_legend().get_texts()
correct_labels = [r"$\it{" + "L. stoechas" + "}$" +" 'Van Gogh's babies'", 
                 r"$\it{" + "L. lanata" + "}$",
                  r"$\it{" + "L. multifida"+ "}$" ,
                  r"$\it{" + "L. stoechas" + "}$" +" 'Kew Red'",
                r"$\it{" + "L. dentata" + "}$" + " var. " + r"$\it{" + " candicans" + "}$",
                  r"$\it{" + "L. dentata" + "}$" +" 'Ploughmen's blue'"]
for i,legend_label in enumerate(legend_labels):
    legend_label.set_text(correct_labels[i])
plt.savefig("Images presentation/Scatterplot robustness (Lavendel good).pdf", format="pdf", bbox_inches='tight')
plt.show()


### Overestimation/Underestimation

In [ ]:
actual_numbers = list(dict_chromosome_number["Actual chromosome number"])
predicted_numbers = list(dict_chromosome_number["Predicted chromosome number"])
difference = []
difference_abs = []
for i in range(len(actual_numbers)):
    actual_number = actual_numbers[i]
    predicted_number = predicted_numbers[i]
    difference_abs.append(abs(actual_number-predicted_number))
    difference.append(actual_number-predicted_number)

In [ ]:
print("Min difference:",np.min(difference_abs))
print("Max difference:",np.max(difference_abs))
print("Mean difference:",np.mean(difference_abs))
print("Median difference:",np.median(difference_abs))

print("Overestimation of the chromosome number:", len([i for i in difference if i < 0]))
print("Underestimation of the chromosome number:", len([i for i in difference if i > 0]))
print("Correct prediction:", len([i for i in difference if i == 0]))

### Mean absolute error

In [ ]:
MAE = sklearn.metrics.mean_absolute_error(list(dict_chromosome_number["Actual chromosome number"]),list(dict_chromosome_number["Predicted chromosome number"]))
print("The mean absolute error is:", MAE)

In [ ]:
difference
print(difference)

In [ ]:
difference.sort()
print(difference)

### Boxplot

In [ ]:
fig, ax = plt.subplots(figsize=(set_size(textwidth)[0], set_size(textwidth)[1]*2))
bp = ax.boxplot(difference, patch_artist=True,
                boxprops=dict(facecolor="steelblue", color="steelblue"),
                whiskerprops=dict(color="steelblue"),
                capprops=dict(color="steelblue"),
                medianprops=dict(color="orange", linewidth=1.5),
                showfliers=True,
                flierprops=dict(markeredgecolor="darkorange"))
bp['boxes'][0].set_facecolor("lightsteelblue")  
ax.yaxis.set_ticks_position('none')
ax.xaxis.set_ticks_position('none')
ax.set_ylim(-15,30)
plt.xticks([]) 
ax.tick_params(axis='both', which='major', labelsize=12)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.grid(color='grey', axis='y', linestyle='-', linewidth=0.25, alpha=0.5)
plt.savefig("Images thesis/Boxplot robustness (Lavendel good).pdf", format="pdf", bbox_inches='tight')

## Good/Bad predictions

In [ ]:
index =  18
 
image = test_images[index]
prediction = test_predictions[index]
post_processed_prediction = post_processed_predictions[index]
n_chromosomes_real = test_ground_truth[i]
n_chromosomes_predicted = len(centroid(post_processed_prediction))
predicted_positions = centroid(post_processed_prediction)
predicted_positions_resized = [(predicted_positions[i][0]-500, predicted_positions[i][1]-100) for i in range(len(predicted_positions))]

legend_label = "Predicted chromosome"
    
plt.figure(figsize=(30,20))
plt.subplot(1,3,1)
plt.imshow(image[100:1300,500:1800], cmap="gray")
plt.title("Annotated image "+str(index), size=30)
plt.axis("off")
plt.subplot(1,3,2)
plt.imshow(prediction[100:1300,500:1800], cmap="nipy_spectral")
plt.title("Prediction", size=30)
plt.axis("off")
plt.subplot(1,3,3)
plt.imshow(post_processed_prediction[100:1300,500:1800], cmap="nipy_spectral")
plt.title("Post-processed prediction", size=30)
plt.scatter(*zip(*predicted_positions_resized), c="w", marker="X", s=200)
plt.axis("off")
plt.tight_layout()
plt.legend([legend_label],fontsize=30, loc=2)

In [ ]:
print(actual_numbers[index])
print(predicted_numbers[index])

In [ ]:
plt.imsave(os.path.join("Images thesis/Predictions robustness/Robustness (Lavendel good)","Original image zoomed "+str(index)+".png"), arr=image[300:1200,800:1700], cmap="gray", format="png")
plt.imsave(os.path.join("Images thesis/Predictions robustness/Robustness (Lavendel good)", "Prediction zoomed "+str(index)+".png"), arr=prediction[300:1200,800:1700], cmap="nipy_spectral", format="png")
plt.imsave(os.path.join("Images thesis/Predictions robustness/Robustness (Lavendel good)","Post-processed prediction zoomed "+str(index)+".png"), arr=post_processed_prediction[300:1200,800:1700], cmap="nipy_spectral", format="png")

In [ ]:
plt.figure(figsize=(21,16.45))
plt.imshow(post_processed_prediction[300:1200,800:1700], cmap="nipy_spectral")
plt.scatter(*zip(*predicted_positions_resized), c="w", marker="X", s=200)
plt.axis("off")
plt.tight_layout()
plt.legend([legend_label],fontsize=50, loc=2)
plt.savefig(os.path.join("Images thesis/Predictions robustness/Robustness (Lavendel good)",  "Predicted chromosomes zoomed "+str(index)+".pdf"), format="pdf", bbox_inches='tight', pad_inches=0)

In [ ]:
index = 18

image = test_images[index]
prediction = test_predictions[index]
post_processed_prediction = post_processed_predictions[index]
n_chromosomes_real = test_ground_truth[index]
n_chromosomes_predicted = len(centroid(post_processed_prediction))
predicted_positions = centroid(post_processed_prediction)
predicted_positions_resized = [(predicted_positions[i][0]-500, predicted_positions[i][1]-100) for i in range(len(predicted_positions))]

legend_label = "Predicted chromosome"

plt.figure(figsize=(30, 20))


ax1 = plt.subplot(1, 3, 1)
ax1.imshow(image[100:1300,500:1800], cmap="gray")
rect = patches.Rectangle((610, 140), 100, 130, linewidth=4, edgecolor="red", facecolor="none")
ax1.add_patch(rect)
ax1.set_title("Annotated image " + str(index), size=30)
ax1.axis("off")

ax2 = plt.subplot(1, 3, 2)
ax2.imshow(prediction[100:1300,500:1800], cmap="nipy_spectral")
rect = patches.Rectangle((610, 140), 100, 130, linewidth=4, edgecolor="red", facecolor="none")
ax2.set_title("Prediction", size=30)
ax2.add_patch(rect)
ax2.axis("off")


ax3 = plt.subplot(1, 3, 3)
ax3.imshow(post_processed_prediction[100:1300,500:1800], cmap="nipy_spectral")
rect = patches.Rectangle((610, 140), 100, 130, linewidth=4, edgecolor="red", facecolor="none")
plt.scatter(*zip(*predicted_positions_resized), c="w", marker="X", s=200)
ax3.set_title("Post-processed prediction", size=30)
ax3.add_patch(rect)
ax3.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(21,16.45))
ax = plt.subplot()
ax.imshow(image[100:1300,500:1800], cmap="gray")
rect = patches.Rectangle((610, 140), 100, 130, linewidth=4, edgecolor="red", facecolor="none")
ax.add_patch(rect)
ax.axis("off")
plt.savefig(os.path.join("Images thesis/Predictions robustness/Robustness (Lavendel good)","Original image zoomed "+str(index)+".pdf"), format="pdf", bbox_inches='tight', pad_inches=0)

plt.figure(figsize=(21,16.45))
ax = plt.subplot()
ax.imshow(prediction[100:1300,500:1800], cmap="nipy_spectral")
rect = patches.Rectangle((610, 140), 100, 130, linewidth=4, edgecolor="red", facecolor="none")
ax.add_patch(rect)
ax.axis("off")
plt.savefig(os.path.join("Images thesis/Predictions robustness/Robustness (Lavendel good)","Prediction zoomed "+str(index)+".pdf"), format="pdf", bbox_inches='tight', pad_inches=0)

plt.figure(figsize=(21,16.45))
ax = plt.subplot()
ax.imshow(post_processed_prediction[100:1300,500:1800], cmap="nipy_spectral")
rect = patches.Rectangle((610, 140), 100, 130, linewidth=4, edgecolor="red", facecolor="none")
plt.scatter(*zip(*predicted_positions_resized), c="w", marker="X", s=200)
plt.legend([legend_label],fontsize=30, loc=2)
ax.add_patch(rect)
ax.axis("off")
plt.savefig(os.path.join("Images thesis/Predictions robustness/Robustness (Lavendel good)","Post-processed prediction zoomed "+str(index)+".pdf"), format="pdf", bbox_inches='tight', pad_inches=0)